In [ ]:
#车牌图像训练

In [34]:
import cv2
import os
import os.path
import math
import random
import sys
import numpy as np
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F

In [35]:
conv=nn.Sequential(
            nn.Conv2d(1,64,kernel_size=3,padding=1), #layer1, inputs single channel,256*128
            nn.ReLU(inplace=True),
            nn.Conv2d(64,64,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(64,128,kernel_size=3,padding=1), #layer2 inputs 64 channel,128*64
            nn.ReLU(inplace=True),
            nn.Conv2d(128,128,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(128,256,kernel_size=3,padding=1), #layer3 inputs 128 channel,64*32
            nn.ReLU(inplace=True),
            nn.Conv2d(256,256,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256,256,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(256,512,kernel_size=3,padding=1), #layer4 inputs 256 channel,32*16
            nn.ReLU(inplace=True),
            nn.Conv2d(512,512,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512,512,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(512,512,kernel_size=3,padding=1), #layer5 inputs 512 channel,16*8
            nn.ReLU(inplace=True),
            nn.Conv2d(512,512,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512,512,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2)
    )

class vgg16train(nn.Module):
    def __init__(self):
        super(vgg16train,self).__init__()
        self.features=conv
        self.classifier=nn.Sequential(
            nn.Dropout(),
            nn.Linear(512 * 8 * 4, 4096),                             #connect: layter: inputs 512, 8*4
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 2048),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(2048, 2)
        )
        #initialize_weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                n = m.weight.size(1)
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()
    def forward(self,x):
        x=self.features(x)
        x=x.view(x.size(0),-1)
        x=self.classifier(x)
        return x

class npModel(nn.Module):
    def __init__(self):
        super(npModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=3,padding=1)   
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3,padding=1) 
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(20*8*16, 100)   #16*32*20
        self.fc2 = nn.Linear(100, 2)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))                                #32*64
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))   #16*32
        x = x.view(-1, 20*8*16)      
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)        

In [36]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count    
        
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)   #pred是索引/位置指示
    pred = pred.t()                                                 #转置
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res        

In [ ]:
#get back picture 生成反例图片/背景图片
OUTPUT_SHAPE=(128,256)
SRCPICDIR='/home/wang/git/deep-anpr/bgs'
DSTPICDIR='/home/wang/git/nppic/nproot/back'
SRC_NUM_PIC=10000
DST_NUM_PIC=500
def generate_bg(num_bg_img):
    found=False
    while not found :
        fname="{}/{:08d}.jpg".format(SRCPICDIR,random.randint(0, SRC_NUM_PIC - 1))
        bg=cv2.imread(fname,cv2.CV_LOAD_IMAGE_GRAYSCALE)
        if(bg.shape[0]>=OUTPUT_SHAPE[0]  and bg.shape[1]>=OUTPUT_SHAPE[1]):
            found=True
    x=random.randint(0,bg.shape[1]-OUTPUT_SHAPE[1])
    y=random.randint(0,bg.shape[0]-OUTPUT_SHAPE[0])
    return bg[y:y+OUTPUT_SHAPE[0],x:x+OUTPUT_SHAPE[1]]
    
if os.path.exists(DSTPICDIR):
    raise IOError,DSTPICDIR+" has been exists" 
os.mkdir(DSTPICDIR)            
i=0
while i<600:
    bg=generate_bg(SRC_NUM_PIC)
    fname=os.path.join(DSTPICDIR,"{:4d}.jpg".format(random.randint(0,9999)))
    cv2.imwrite(fname,generate_bg(SRC_NUM_PIC))
    i=i+1

In [ ]:
#mean and variance 得到均值和方差
TRAINDIR='/home/wang/git/nppic/meanval'
f=[]
for parent,dirnames,filenames in os.walk(TRAINDIR):
    for filename in filenames:
        f.append(filename)
        
img=np.ndarray((len(f),128,256))
for i,fname in enumerate(f):
    img[i]=cv2.imread(os.path.join(TRAINDIR,fname),cv2.CV_LOAD_IMAGE_GRAYSCALE)
mean=np.mean(img)
var=np.var(img)
print mean,var
#another method, equal to mean/var of numpy
#mean=0
#cnt=0
#i=np.transpose(img,(1,2,0))    
#for r,row in enumerate(i):
#    for c,column in enumerate(row):
#        for idx,dot in enumerate(column):
#            mean=mean+dot
#            cnt=cnt+1
#mean=mean/cnt            
#v=0
#cnt=0
#for r,row in enumerate(i):
#    for c,column in enumerate(row):
#        for idx,dot in enumerate(column):
#            v=v+(dot-mean)**2
#            cnt=cnt+1

In [ ]:
def make_scaled_ims(im, min_shape):
    ratio = 1. / 2 ** 0.5
        yield cv2.resize(im, (shape[1], shape[0]))
        

In [37]:
def gray_loader(path):#input 128*256
    return cv2.imread(path,cv2.CV_LOAD_IMAGE_GRAYSCALE)
    #img=cv2.imread(path,cv2.CV_LOAD_IMAGE_GRAYSCALE)
    #return cv2.resize(img,(64,32))

In [38]:
batch_size=5
data_transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((107.897212036,), (3893.57887653,)),
                             ])
npset = datasets.ImageFolder(root='/home/wang/git/nppic/nproot', 
                             transform=data_transform,loader=gray_loader)
nploader = torch.utils.data.DataLoader(npset, batch_size=batch_size, shuffle=True, 
                                       num_workers=1)  #train
npvalset=datasets.ImageFolder(root='/home/wang/git/nppic/npval', 
                              transform=data_transform,loader=gray_loader)
npvalloader=torch.utils.data.DataLoader(npvalset, batch_size=batch_size, shuffle=False, 
                                        num_workers=1) #validate
print 'over'

over


In [39]:
model=vgg16train()
#model=npModel()
#model.cuda()
criterion = nn.CrossEntropyLoss() # use a Classification Cross-Entropy loss
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
print 'over'

over


In [ ]:
#Train
losses=AverageMeter()
model.train()
for epoch in range(4):
    losses.reset()
    for _, (datas,targets) in enumerate(nploader):
        datas=torch.unsqueeze(datas,1)
        datas_var,targets_var=torch.autograd.Variable(datas),torch.autograd.Variable(targets)
        #datas_var=datas_var.cuda()
        #targets_var=targets_var.cuda()
        
        optimizer.zero_grad()
        outputs_var = model(datas_var)
        loss = criterion(outputs_var, targets_var)
        loss.backward()        
        optimizer.step()
        #loss
        losses.update(loss.data[0],datas_var.data.size(0))  # datas size(0) is batch_size of input
        if losses.count % 100 == 0:
            print ('[%d, %4d] currnet los: %.5f, ave loss: %.5f' %(epoch+1,losses.count,loss.data[0], losses.avg)) 
 

In [ ]:
torch.save(model.state_dict(),'./plate.weight')

In [ ]:
#validate
prediction=AverageMeter()
model.eval()
for _, (datas,targets) in enumerate(npvalloader):
    datas=torch.unsqueeze(datas,1)
    datas_var,targets_var=torch.autograd.Variable(datas),torch.autograd.Variable(targets)
    #datas_var=datas_var.cuda()
    #targets_var=targets_var.cuda()    
    optimizer.zero_grad()
    outputs_var = model(datas_var)
    prec=accuracy(outputs_var.data,targets_var.data)   #40% will return 40
    prediction.update(prec[0].cpu().numpy()[0],datas_var.data.size(0))
    
#print(' accuracy is{val.avg:%.2f}'.format(val=prediction))
print ('accuracy is: %.2f' %(prediction.avg)) 


In [ ]:
sm=nn.Softmax()
d=sm(outputs_var)
print d
print '------------'
print torch.max(d,1)
#print '------------'
#print datas_var.data.size()
#print '------------'
#print outputs_var.data
#print '------------'
#print torch.max(outputs_var.data,1)
#print '------------'
#print targets_var.data


In [ ]:
#test
#原图传入，如果概率小于50%，则认为不存在
#按照上一次图的70%截取ROI，步长为短边×(1-30%)×1/4进行滑动，生成ROI集合，
#        进行检测，取概率最大的一个进行迭代，直到
#  1. 某一次检测的所有ROI集合中成员检测概率都不超过50%
TESTPICDIR='/home/wang/git/nppic/su_np_gray/20170221_143111_gray.jpg'
OUTPUTDIR='/home/wang/git/nppic'
################
#intput,simgle channel
#the returning: 
# img list
#or none if 70% of img shape[0] < OUT_SHAPE[0] or  img shape[1] < OUT_SHAPE[1]
#or step < 2
def get_roi_set(img):
    rect_y=int(img.shape[0]*0.7)
    rect_x=int(img.shape[1]*0.7)
    if rect_y<OUTPUT_SHAPE[0] or rect_x<OUTPUT_SHAPE[1]:
        return None
    step=int(min(img.shape[0]*0.3,img.shape[1]*0.3)//4)
    if step<2:  
        return None
    
    cordinate_x=0
    cordinate_y=0 #ROI右上基准
    res=[]
    while cordinate_y+rect_y < img.shape[0]:
        cordinate_x=0
        while cordinate_x+rect_x < img.shape[1]:
            res.append(img[cordinate_y:cordinate_y+rect_y, cordinate_x:cordinate_x+rect_x].copy())
            cordinate_x += step
        #the  right column
        if (cordinate_x+rect_x-img.shape[1]) < rect_y-128 :   #there are 128 pixels at least in the right rest
            tt=np.zeros((rect_y,rect_x),dtype=np.uint8)
            tt[0:rect_y,0: img.shape[1]-cordinate_x]=img[cordinate_y:cordinate_y+rect_y, cordinate_x:]
            res.append(tt.copy())
        cordinate_y += step
    #the bottom row    
    if (cordinate_y+rect_y-img.shape[0]) < rect_x-64: #there are 64 pixels at least in the bottom rest
        cordinate_x=0
        while cordinate_x+rect_x < img.shape[1]:
            tt=np.zeros((rect_y,rect_x),dtype=np.uint8)
            tt[0:img.shape[0]-cordinate_y, 0:rect_x]=img[cordinate_y:, cordinate_x:cordinate_x+rect_x]
            res.append(tt.copy())
            cordinate_x+=step
        #the buttom right corner
        if (cordinate_x+rect_x-img.shape[1]) < rect_y-128 :
            tt=np.zeros((rect_y,rect_x),dtype=np.uint8)
            tt[0:img.shape[0]-cordinate_y, 0: img.shape[1]-cordinate_x]=img[cordinate_y:, cordinate_x:]
            res.append(tt.copy())
    return res

img=cv2.imread(TESTPICDIR,cv2.CV_LOAD_IMAGE_GRAYSCALE)
res=get_roi_set(img)
for i,img in enumerate(res):
    cv2.imwrite(os.path.join(OUTPUTDIR,"{:04d}.jpg".format(i)),img)

In [ ]:
#mean and variance
TRAINDIR='/home/wang/git/nppic/nproot/plate'
f=[]
for parent,dirnames,filenames in os.walk(TRAINDIR):
    for filename in filenames:
        f.append(filename)
img=np.ndarray((len(f),128,256))
for i,fname in enumerate(f):
    img[i]=cv2.imread(os.path.join(TRAINDIR,fname),cv2.CV_LOAD_IMAGE_GRAYSCALE)

img=img/255  #[0,1]

mean=np.mean(img)   #-0.18262765251608459
var=np.var(img)             #0.21571192247129692
    

In [ ]:
#获取直方图，单通道
def getHist(img):
    return cv2.calcHist([img], [0], None, [256], [0.0,255.0])  

#生产直方图,单通道
def calcAndDrawHist(image, color=[255,255,255]): 
    hist= cv2.calcHist([image], [0], None, [256], [0.0,255.0])    
    minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(hist)    
    histImg = np.zeros([256,256,3], np.uint8)    
    hpt = int(0.9* 256);    
        
    for h in range(256):    
        intensity = int(hist[h]*hpt/maxVal)    
        cv2.line(histImg,(h,256), (h,256-intensity), color)    
            
    return histImg;#生产直方图,单通道

#生成直方图，三通道，折线方式
def calcAndDrawHistPolyline(img): 
    h = np.zeros((256,256,3)) #创建用于绘制直方图的全0图像    
         
    bins = np.arange(256).reshape(256,1) #直方图中各bin的顶点位置    
    color = [ (255,0,0),(0,255,0),(0,0,255) ] #BGR三种颜色    
    for ch, col in enumerate(color):    
        originHist = cv2.calcHist([img],[ch],None,[256],[0,256])    
        cv2.normalize(originHist, originHist,0,255*0.9,cv2.NORM_MINMAX)    
        hist=np.int32(np.around(originHist))    
        pts = np.column_stack((bins,hist))    
        cv2.polylines(h,[pts],False,col)    
         
    h=np.flipud(h)
    return h

def imshow(img):
    plt.imshow(img)
    plt.show()
    

In [ ]:
img=cv2.imread(imgfile)#,cv2.CV_LOAD_IMAGE_GRAYSCALE) #img: [height, width, channels]
x=img.transpose(2,0,1)
#h=calcAndDrawHist(x[0]) #hist of channels 0 
#imshow(h)
#原图
#imshow(img)
#h=calcAndDrawHist(img)
#imshow(h)
#直方图均衡化
b=cv2.equalizeHist(x[0])
g=cv2.equalizeHist(x[1])
r=cv2.equalizeHist(x[2])
bgr=np.array((b,g,r))
print bgr.shape
img=bgr.transpose(1,2,0)  #channel,height,width -> height,width,channel
#imshow(img)
#点运算--二值化，灰度线性变换（改变对比度），灰度拉伸（过曝/不足），灰度均衡
#f(dot)=k2*dot.blue+dot.green/k1+dot.red/k1
def dotcalc((b,g,r),k1,k2):
    return k2*b+g/k1+r/k1
grayimg=np.zeros((img.shape[0],img.shape[1]),np.uint8)
for h,row in enumerate(img):
    for w,dot in enumerate(row):
        b,g,r=dot
        grayimg[h][w]=dotcalc((b,g,r),0.8,0.1)
imshow(grayimg)        

In [ ]:
def imshow(img):
    plt.imshow(img)
    plt.show()

In [ ]:
#http://blog.csdn.net/honghu549599aaa/article/details/51275349
def colorDetect(image,option=0):
    img = cv2.imread(image)
    colorImage = img.copy()
    _colorImage = img.copy()
    hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    #cv2.imshow("hsv",hsv)
    #高斯模糊
    img = cv2.GaussianBlur(img,(5,5),0)
    #cv2.imshow("hsv",hsv)
    # 设定蓝色的阈值
    if(option == 0):
        lower=np.array([100,50,50])
        upper=np.array([140,255,255])
    else:
        #黄色
        lower=np.array([15,50,50])
        upper=np.array([40,255,255])
    # 根据阈值构建掩模
    mask=cv2.inRange(hsv,lower,upper)
    # 对原图像和掩模进行位运算
    res=cv2.bitwise_and(img,img,mask=mask)
    gray = cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)
    #二值化
    ret,thresh1 = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    #cv2.imshow('gray',gray)
    #闭操作
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(17, 3))
    closed = cv2.morphologyEx(thresh1, cv2.MORPH_CLOSE, kernel)
    #cv2.imshow('closed',closed)
    (cnts, _) = cv2.findContours(closed.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    #boxes = choose(cnts)
    cv2.drawContours(img,cnts,-1,(0,0,255),1)
    return img
    imgRs = []
    i = 0
    for cnt in cnts:
        rect = cv2.minAreaRect(cnt)
        x,y,w,h = cv2.boundingRect(cnt)
        if(w<50 or h < 15 or w>h < 1.0):
            continue
        #cv2.rectangle(_colorImage,(x,y),(x+w,y+h),(0,255,0),1)
        #imgCrop = _colorImage[y:y+h,x:x+w]
        imgRs.append((x,y,w,h,rect[2]))
        rs = img[y:y+h,x:x+w]
        #cv2.imshow("============="+str(name),rs)
    #cv2.drawContours(_colorImage, [_box], -1, (0,0,255), 1)
    #cv2.imshow("_colorImage",_colorImage)
    return imgRs

In [ ]:
#npcolor='/home/wang/git/nppic/su_np_color/20170218_091712.jpg'
#npcolor='/home/wang/git/nppic/su_np_color/20170220_141422.jpg'
npcolor='/home/wang/git/nppic/su_np_color-256x128/00000113_0A9U2W1_1.png'

imnp=colorDetect(npcolor)
imshow(imnp)
cv2.imwrite('/home/wang/git/nppic/20170218_091712.png',imnp)